In [1]:
import pickle
import random
import numpy as np

In [2]:
train_file = "../Data/tmall/taobao_small_cate_train.pickle"
train_file_writer = open(train_file, "rb")
train_list = pickle.load(train_file_writer)

test_file = "../Data/tmall/taobao_small_cate_test.pickle"
test_file_writer = open(test_file, "rb")
test_list = pickle.load(test_file_writer)

In [3]:
len_list = []
train_user_num = len(train_list)
for train_i in range(train_user_num):
    user_list = train_list[train_i]
    len_list.append(len(user_list))
print(np.mean(len_list), np.max(len_list), np.min(len_list))

46.80525 184 1


In [4]:
np.zeros((3, 3))

array([[0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.]])

In [5]:
second_order_matrix = np.zeros((5869, 5869))
first_order_matrix = np.zeros((5869, 5869))

# categoryList = []

for user_index in range(train_user_num):
    user_list = train_list[user_index]
    user_action_len = len(user_list)
        
    last_action_category = -1
    for user_action_index in range(2, user_action_len):
        last_action_category = user_list[user_action_index-2]
        user_action_category = user_list[user_action_index]
        second_order_matrix[last_action_category, user_action_category] += 1.0
        
    for user_action_index in range(1, user_action_len):
        last_action_category = user_list[user_action_index-1]
        user_action_category = user_list[user_action_index]
        first_order_matrix[last_action_category, user_action_category] += 1.0
        
#         categoryList.append(user_action_category)
        
#         if user_action_index == 0:
            
#             last_action_category = user_action_category
#             continue
        
#         last_action_category = int(last_action_category)
#         user_action_category = int(user_action_category)
#         first_order_matrix[last_action_category, user_action_category] += 1.0
        
#         last_action_category = user_action_category
               

In [14]:
test_user_num = len(test_list)

topk = 1

weight = 0.8

correct_num_topk = 0
total_num = 0

for user_index in range(test_user_num):
    user_list = test_list[user_index]
    user_action_len = len(user_list)
    
    last_action_category = -1
    for user_action_index in range(2, user_action_len):
        user_action_category = user_list[user_action_index]
        
        if user_action_category >= len(second_order_matrix):
            continue
        
        last_action_category_second = user_list[user_action_index-2]
        if last_action_category_second >= len(second_order_matrix):
            continue
            
        last_action_category_second = int(last_action_category_second)
        user_action_category = int(user_action_category)
        category_score_list_second = second_order_matrix[last_action_category_second, :]
        
        last_action_category_first = user_list[user_action_index-1]
        if last_action_category_first >= len(first_order_matrix):
            continue
        last_action_category_first = int(last_action_category_first)
        user_action_category = int(user_action_category)
        
        category_score_list_first = first_order_matrix[last_action_category_first, :]
        
        category_score_list = weight*category_score_list_second/np.sum(category_score_list_second)+(1-weight)*category_score_list_first/np.sum(category_score_list_first)
        
        last_action_sum = np.sum(category_score_list)
        if last_action_sum == 0:
            print(last_action_category, user_action_category, "zero")
        
        category_score_list = category_score_list/last_action_sum
        
        pred = (-np.array(category_score_list)).argsort()[:topk]
        if user_action_category in pred:
            correct_num_topk += 1.0
        total_num += 1.0
        
#         last_action_category = user_action_category
        
print("correct num topk", correct_num_topk)
print("total num topk", total_num)      

/zf15/rc7ne/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:37: RuntimeWarning: invalid value encountered in true_divide


correct num topk 171902.0
total num topk 905862.0


In [15]:
recall_topk = correct_num_topk/total_num
print("recall@%d"%topk, recall_topk)

recall@1 0.1897662116304691


In [11]:
recall_topk = correct_num_topk/total_num
print("recall@%d"%topk, recall_topk)

recall@1 0.18814455181915127


In [9]:
recall_topk = correct_num_topk/total_num
print("recall@%d"%topk, recall_topk)

recall@1 0.1700579116907432


In [39]:
first_order_matrix[:10, :10]

array([[0.000e+00, 1.427e+03, 3.606e+03, 8.510e+02, 2.480e+02, 4.203e+03,
        8.000e+00, 2.523e+03, 1.469e+03, 6.064e+03],
       [1.444e+03, 0.000e+00, 6.140e+02, 2.960e+02, 1.090e+02, 4.530e+02,
        0.000e+00, 8.510e+02, 5.560e+02, 9.970e+02],
       [3.641e+03, 6.320e+02, 0.000e+00, 5.360e+02, 1.350e+02, 1.112e+03,
        5.000e+00, 1.741e+03, 8.330e+02, 2.916e+03],
       [8.500e+02, 2.710e+02, 5.450e+02, 0.000e+00, 1.510e+02, 3.320e+02,
        1.470e+02, 1.672e+03, 5.140e+02, 1.564e+03],
       [2.420e+02, 8.700e+01, 1.220e+02, 1.690e+02, 0.000e+00, 1.100e+02,
        3.000e+00, 5.070e+02, 1.370e+02, 4.320e+02],
       [4.104e+03, 4.390e+02, 1.116e+03, 3.190e+02, 1.400e+02, 0.000e+00,
        4.000e+00, 1.027e+03, 8.030e+02, 2.213e+03],
       [1.200e+01, 2.000e+00, 2.000e+00, 1.400e+02, 2.000e+00, 3.000e+00,
        0.000e+00, 1.800e+01, 4.000e+00, 1.700e+01],
       [2.399e+03, 8.430e+02, 1.797e+03, 1.808e+03, 5.170e+02, 1.018e+03,
        1.300e+01, 0.000e+00, 1.419e+

In [29]:
a = np.array([1, 10, 3, 4])
print((-a).argsort()[:2])

[1 3]
